In [85]:
# Cell 1: Imports and core transformation functions

import pandas as pd

def load_survey_sheet(excel_path: str, sheet_name: str) -> pd.DataFrame:
    """
    Load one sheet from the Excel survey file.
    """
    return pd.ExcelFile(excel_path).parse(sheet_name)

def transform_survey_df(
    df: pd.DataFrame,
    aggregation_level: str,
    id_cols: list[str]
) -> pd.DataFrame:
    """
    Turn the wide-format survey df into long-format matching the MBTA CSV schema.
    
    - id_cols: list of columns to keep as identifiers (e.g. ['Mode','Line','Station'])
    - aggregation_level: 'Station' or 'Line'
    """
    # 1. select only the percent columns
    value_cols = [
        c for c in df.columns
        if c not in id_cols and 'count' not in c.lower()
    ]
    
    # 2. melt to long format
    long = df.melt(
        id_vars=id_cols,
        value_vars=value_cols,
        var_name='measure_category',
        value_name='weighted_percent'
    ).dropna(subset=['weighted_percent'])
    
    # 3. split measure_group / category
    long[['measure_group','category']] = (
        long['measure_category']
        .str.split(':', n=1, expand=True)
        .apply(lambda s: s.str.strip())
    )
    
    # 4. fill static and renamed fields
    long['aggregation_level'] = aggregation_level
    long['service_mode']     = long[id_cols[0]]
    long['reporting_group']  = long[id_cols[-1]]
    long['measure']          = long['measure_group']
    long['ObjectId']         = range(1, len(long) + 1)
    
    # 5. select & order final columns
    return long[[
        'aggregation_level',
        'service_mode',
        'reporting_group',
        'measure_group',
        'measure',
        'category',
        'weighted_percent',
        'ObjectId'
    ]]

def sort_survey_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Ensure each reporting_group’s block is contiguous,
    with measure_groups & categories ordered within.
    """
    # preserve original appearance order
    df['reporting_group'] = pd.Categorical(
        df['reporting_group'],
        categories=df['reporting_group'].unique(),
        ordered=True
    )
    df['measure_group'] = pd.Categorical(
        df['measure_group'],
        categories=df['measure_group'].unique(),
        ordered=True
    )
    return df.sort_values(
        by=['reporting_group','measure_group','category']
    ).reset_index(drop=True)


In [87]:
# Cell 2: Main execution

def main():
    excel_path = 'MBTA systemwide survey results by station and line.xlsx'
    
    # Process Station percentages
    station_df = load_survey_sheet(excel_path, 'Station percentages')
    station_long = transform_survey_df(
        station_df,
        aggregation_level='Station',
        id_cols=['Mode','Line','Station']
    )
    station_long = sort_survey_df(station_long)
    station_long.to_csv('Transformed_MBTA_Station_Percentages.csv', index=False)
    
    # Process Line percentages
    line_df = load_survey_sheet(excel_path, 'Line percentages')
    line_long = transform_survey_df(
        line_df,
        aggregation_level='Line',
        id_cols=['Mode','Line']
    )
    line_long = sort_survey_df(line_long)
    line_long.to_csv('Transformed_MBTA_Line_Percentages.csv', index=False)
    
    print("All sheets transformed and saved.")

if __name__ == '__main__':
    main()


All sheets transformed and saved.
